## 1.Importing Data To .CSV File

In [ ]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
 os.makedirs(housing_path, exist_ok=True)
 tgz_path = os.path.join(housing_path, "housing.tgz")
 urllib.request.urlretrieve(housing_url, tgz_path)
 housing_tgz = tarfile.open(tgz_path)
 housing_tgz.extractall(path=housing_path)
 housing_tgz.close()

In [ ]:
import pandas as pd
import numpy as np
def load_housing_data(housing_path=HOUSING_PATH):
 csv_path = os.path.join(housing_path, "housing.csv")
 return pd.read_csv(csv_path)

## 2.Analyzing the data
let's first import our data and have a quick look at the first 5 columns using head method 

In [ ]:
fetch_housing_data() #fetching data
housing = load_housing_data()
housing.head()

Now let's see infos about our data

In [ ]:
housing.info()

We remarque that some columns contain non numerical values such as ocean_proximity , let's have a look at how many values does this column may contain

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

Another way to analyze the data is by plotting into histograms

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
housing.hist(bins=50, figsize=(20,15))
plt.show()

## 3.Creating The test set 
Now we need to create a test set , so we will build a function that splits our data into a train set and a test set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

## 4.Looking for Correlations

In [ ]:
corr_matrix = housing.corr(numeric_only=True)

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

If correlation is close to 1 , it means if a y goes up the, x also goes up , -1 is the opposite and for 0 there is no linear relation between y and x

Another way to check for correlation between attributes is to use the pandas
scatter_matrix() function

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms",
 "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

The most promising attribute to predict the median house value is the median
income, so let’s zoom in on their correlation scatterplot

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
 alpha=0.1)


This plot reveals a few things. First, the correlation is indeed very strong; you can
clearly see the upward trend, and the points are not too dispersed. Second, the price
cap that we noticed earlier is clearly visible as a horizontal line a $500,000. But this
plot reveals other less obvious straight lines: a horizontal line aroud $450,000,
another aro3d $350,000, perhaps one around $280,000, and a few more below that.
You may want to try removing the corresponding districts to prevent your algorithms
from learning to reproduce these data quirks.

Since Now we analyzed the data , we may create some attributes that may be more interesting for our case 

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]


Now let's see their correlation 

In [ ]:
corr_matrix = housing.corr(numeric_only=True)

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

Hey, not bad! The new bedrooms_per_room attribute is much more correlated with
the median house value than the total number of rooms or bedrooms. Apparently
houses with a lower bedroom/room ratio tend to be more expensive. The number of
rooms per household is also more informative than the total number of rooms in a
district—obviously the larger the houses, the more expensive they ar

## Prepare the Data for Machine Learning Algorithms

Let’s also separate the predictors and the labels, since we don’t necessarily want to
apply the same transformations to the predictors and the target values

In [ ]:
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()
test_housing = test_set.drop("median_house_value", axis=1)
test_housing_labels = test_set["median_house_value"].copy()

Let's replace missing values by the median of that column , that's what we will do with $ total bedrooms $ feature

In [ ]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)
median

Scikit-Learn provides a handy class to take care of missing values: SimpleImputer.
Here is how to use it. First, you need to create a SimpleImputer instance, specifying
that you want to replace each attribute’s missing values with the median of that
attribute:

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

Since the median can only be computed on numerical attributes, you need to create a
copy of the data without the text attribute ocean_proximity

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)
housing_num

Now you can fit the imputer instance to the training data using the fit() method and the result will be stored in statistics method:

In [ ]:
imputer.fit(housing_num)
housing_num

The imputer has simply computed the median of each attribute and stored the result
in its statistics_ instance variable.

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values


Now you can use this “trained” imputer to transform the training set by replacing
missing values with the learned medians:

In [ ]:
X = imputer.transform(housing_num)
X

The result is a plain NumPy array containing the transformed features. If you want to
put it back into a pandas DataFrame, it’s simple:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
 index=housing_num.index)
housing_tr

# Handling Text and Categorical Attributes

Let's visualize some of the ocean_proximity column containing categorical data

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

Let's use ordinal encoding to convert text values to numerical values 

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

You can get the list of categories using the categories_ instance variable. It is a list
containing a 1D array of categories for each categorical attribute (in this case, a list
containing a single array since there is just one categorical attribute):

In [ ]:
ordinal_encoder.categories_

One issue with this representation is that ML algorithms will assume that two nearby
values are more similar than two distant values. This may be fine in some cases (e.g.,
for ordered categories such as “bad,” “average,” “good,” and “excellent”), but it is obvi‐
ously not the case for the ocean_proximity column (for example, categories 0 and 4
are clearly more similar than categories 0 and 1). To fix this issue, a common solution
is to create one binary attribute per category: one attribute equal to 1 when the cate‐
gory is “<1H OCEAN” (and 0 otherwise), another attribute equal to 1 when the cate‐
gory is “INLAND” (and 0 otherwise), and so on. This is called ** one-hot encoding, **
because only one attribute will be equal to 1 (hot), while the others will be 0 (cold).
The new attributes are sometimes called dummy attributes. Scikit-Learn provides a
OneHotEncoder class to convert categorical values into one-hot vectors:20

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

## Feature Scaling

There are two common ways to get all attributes to have the same scale: min-max (normalization)
scaling and standardization. for min max , it's simple we substract the min value and divide by the max value all of our example. Whereas standarization first subtracts the mean value (so standardized values 
always have a zero mean), and then it divides by the standard deviation so that th 
resulting distribution has unit varian.ce

## Transformation Pipelines

Pipelines in scikitlearn help us gather all the transofmation we want to apply to the data in the same order . SO let's create a transformation pipeline for our numerical features!

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('std_scaler', StandardScaler()),
 ])
housing_num_tr = num_pipeline.fit_transform(housing_num)

There is a way to both treat numerical and categorical data in one shot 

In [ ]:
from sklearn.compose import ColumnTransformer
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]
full_pipeline = ColumnTransformer ([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs),
 ])
housing_prepared = full_pipeline.fit_transform(housing)
test_prepared = full_pipeline.fit_transform(test_housing)

## Select and Train a Model

now it's time to select a model to train it on our cleaned data

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
housing_predictions = lin_reg.predict(test_prepared)
lin_mse = mean_squared_error(test_housing_labels, housing_predictions)
lin_r2 = r2_score(test_housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse , lin_mse, lin_r2